## SAY SOMETHING HERE

A company-year long format xt panel data table, 2005-2016 
 n = 287,829 observations (46,412 firms)
	(third as many companies in 2016 as in earlier years)
 - ID variables	comp_id	(numerical) company identifier
- year	(calendar) year

 other important variables
 
- exit_year	year of exit (missing if still in business on 2016.12.31) 
- sales		sales in year (EUR)
- curr_assets	current assets at end of year (EUR)
- ind2		2-digit NACE industry code

DEFAULT =  Existed in 2014 (sales > 0), but did not exists in 2015 (sales is 0 or missing)

In [30]:
import pandas as pd
import numpy as np
from plotnine import *
import warnings
warnings.filterwarnings('ignore')

In [65]:
data = pd.read_csv('cs_bisnode_panel.zip')
data

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1.001034e+06,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1.001034e+06,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1.001034e+06,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1.001034e+06,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1.001034e+06,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287824,4.641209e+11,2011-01-01,2011-12-31,NaN,0.000000,1807.407349,3.703704,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287825,4.641209e+11,2012-01-01,2012-12-31,NaN,0.000000,1518.518555,96.296295,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287826,4.641209e+11,2013-01-01,2013-12-31,NaN,0.000000,988.888916,77.777779,0.0,0.0,0.0,...,NaN,NaN,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287827,4.641209e+11,2014-01-01,2014-12-31,NaN,0.000000,644.444458,233.333328,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN


In [66]:
data = data.loc[data['ind2'] == 26]
data
#data for specific industry

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
960,6.538183e+06,2005-01-01,2005-12-31,NaN,792.592590,6237.037109,348.148163,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
961,6.538183e+06,2006-01-01,2006-12-31,NaN,196.296295,6007.407227,451.851837,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
962,6.538183e+06,2007-01-01,2007-12-31,NaN,233.333328,9007.407227,2637.037109,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
963,6.538183e+06,2008-01-01,2008-12-31,NaN,422.222229,7707.407227,1651.851807,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
964,6.538183e+06,2009-01-01,2009-12-31,NaN,1525.925903,18196.296875,8714.814453,18.518518,0.0,-18.518518,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287439,4.628231e+11,2012-01-01,2012-12-31,NaN,359.259247,21662.962891,19481.482422,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.333333
287440,4.628231e+11,2013-01-01,2013-12-31,NaN,3159.259277,15488.888672,21440.740234,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.298611
287441,4.628231e+11,2014-01-01,2014-12-31,NaN,2581.481445,70059.257812,74911.109375,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.486111
287442,4.628231e+11,2015-01-01,2015-12-31,NaN,2177.777832,46062.964844,49844.445312,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.562500


In [33]:
#cleaning dataset:

#x = amort



#imputing missing amort values with median (less affected by outliers)
#creating flag_variable for missing values and imputed values
data.amort.isnull().sum()
data['amort_missing_flag'] = data.amort.isnull().astype(int)
data['amort_imputed_flag'] = data.amort.isnull().astype(int)
data.fillna({'amort':data['amort'].median()}, inplace = True)



amort
0.000000        2478
1818.518555      293
222.222229        29
140.740738        27
103.703697        27
                ... 
63118.519531       1
62737.035156       1
74770.367188       1
98896.296875       1
5362.962891        1
Name: count, Length: 5363, dtype: int64

### Checking to see how many com

In [107]:
#trying to see how many companies defaulted, first by creating 2014_data
data.sort_values(by = ['comp_id', 'year'], inplace = True)
data_2014 = data.loc[data['year'] == 2014]
data_2014['company_existed_2014'] = 0
data_2014.loc[(data_2014['sales'] > 0), 'company_existed_2014'] = 1
data_2014

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg,company_existed_2014
969,6.538183e+06,2014-01-01,2014-12-31,NaN,1018.518494,17022.222656,3040.740723,0.0,0.0,0.0,...,Domestic,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN,1
1128,8.416055e+06,2014-01-01,2014-12-31,NaN,425.925934,105740.742188,6918.518555,0.0,0.0,0.0,...,Domestic,2651.0,26.0,2.0,1,Central,1995-08-28,NaN,0.083333,1
1342,1.089394e+07,2014-07-24,2014-12-31,NaN,0.000000,30614.814453,128055.546875,0.0,0.0,0.0,...,Domestic,2651.0,26.0,2.0,1,Central,2004-12-13,NaN,NaN,0
1467,1.242838e+07,2014-01-01,2014-12-31,NaN,0.000000,0.000000,9277.777344,0.0,0.0,0.0,...,Domestic,2660.0,26.0,2.0,2,Central,1991-06-27,NaN,0.083333,1
1593,1.571155e+07,2014-01-01,2014-12-31,NaN,0.000000,2477.777832,0.000000,0.0,0.0,0.0,...,NaN,2630.0,26.0,2.0,1,Central,2001-09-01,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286758,4.604901e+11,2014-01-01,2014-12-31,NaN,0.000000,33.333332,585.185181,0.0,0.0,0.0,...,Domestic,2640.0,26.0,2.0,2,East,2000-06-14,NaN,NaN,0
286890,4.610112e+11,2014-01-01,2014-12-31,NaN,429.629639,58781.480469,2651.851807,0.0,0.0,0.0,...,Domestic,2620.0,26.0,2.0,3,Central,1997-12-10,NaN,0.166667,1
287204,4.620594e+11,2014-01-01,2014-12-31,NaN,1111.111084,68274.070312,16211.111328,0.0,0.0,0.0,...,Domestic,2630.0,26.0,2.0,1,Central,1996-12-30,NaN,0.250000,1
287272,4.623300e+11,2014-01-01,2014-12-31,NaN,0.000000,17192.591797,7325.925781,0.0,0.0,0.0,...,mix,2611.0,26.0,2.0,2,East,2010-12-15,NaN,0.166667,1


In [111]:
#trying to see how many companies defaulted, secondly by creating 2015_data
data_2015 = data.loc[data['year'] == 2015]
data_2015['company_not_existed_2015'] = 0
data_2015.loc[(data_2015['sales'].isna())|(data_2015['sales'] == 0), 'company_not_existed_2015'] = 1
data_2015

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg,company_not_existed_2015
970,6.538183e+06,2015-01-01,2015-12-31,NaN,1029.629639,18944.445312,2470.370361,0.0,0.0,0.0,...,Domestic,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN,0
1129,8.416055e+06,2015-01-01,2015-12-31,NaN,529.629639,86151.851562,2555.555664,0.0,0.0,0.0,...,Domestic,2651.0,26.0,2.0,1,Central,1995-08-28,NaN,NaN,0
1343,1.089394e+07,2015-01-01,2015-12-31,NaN,0.000000,30614.814453,128055.546875,0.0,0.0,0.0,...,Domestic,2651.0,26.0,2.0,1,Central,2004-12-13,NaN,NaN,1
1468,1.242838e+07,2015-01-01,2015-12-31,NaN,0.000000,118.518517,11081.481445,0.0,0.0,0.0,...,Domestic,2660.0,26.0,2.0,2,Central,1991-06-27,NaN,0.083333,0
1508,1.375798e+07,2015-12-10,2015-12-31,NaN,0.000000,29200.000000,25225.925781,0.0,0.0,0.0,...,Domestic,2630.0,26.0,2.0,2,Central,2000-05-07,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286759,4.604901e+11,2015-01-01,2015-12-31,NaN,0.000000,14.814815,711.111084,0.0,0.0,0.0,...,Domestic,2640.0,26.0,2.0,2,East,2000-06-14,NaN,NaN,1
286891,4.610112e+11,2015-01-01,2015-12-31,NaN,411.111115,54511.109375,4548.147949,0.0,0.0,0.0,...,Domestic,2620.0,26.0,2.0,3,Central,1997-12-10,NaN,0.242424,0
287205,4.620594e+11,2015-01-01,2015-12-31,NaN,1159.259277,73688.890625,20029.628906,0.0,0.0,0.0,...,Domestic,2630.0,26.0,2.0,1,Central,1996-12-30,NaN,0.222222,0
287273,4.623300e+11,2015-01-01,2015-12-31,NaN,0.000000,22740.740234,6881.481445,0.0,0.0,0.0,...,mix,2611.0,26.0,2.0,2,East,2010-12-15,NaN,0.166667,0


In [122]:
#merge the two together to see how many defaulted
yo = pd.merge(data_2014, data_2015, on = 'comp_id')
yo

,comp_id,begin_x,end_x,COGS_x,amort_x,curr_assets_x,curr_liab_x,extra_exp_x,extra_inc_x,extra_profit_loss_x,...,origin_y,nace_main_y,ind2_y,ind_y,urban_m_y,region_m_y,founded_date_y,exit_date_y,labor_avg_y,company_not_existed_2015
0,6.538183e+06,2014-01-01,2014-12-31,NaN,1018.518494,17022.222656,3040.740723,0.0,0.0,0.0,...,Domestic,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN,0
1,8.416055e+06,2014-01-01,2014-12-31,NaN,425.925934,105740.742188,6918.518555,0.0,0.0,0.0,...,Domestic,2651.0,26.0,2.0,1,Central,1995-08-28,NaN,NaN,0
2,1.089394e+07,2014-07-24,2014-12-31,NaN,0.000000,30614.814453,128055.546875,0.0,0.0,0.0,...,Domestic,2651.0,26.0,2.0,1,Central,2004-12-13,NaN,NaN,1
3,1.242838e+07,2014-01-01,2014-12-31,NaN,0.000000,0.000000,9277.777344,0.0,0.0,0.0,...,Domestic,2660.0,26.0,2.0,2,Central,1991-06-27,NaN,0.083333,0
4,1.571155e+07,2014-01-01,2014-12-31,NaN,0.000000,2477.777832,0.000000,0.0,0.0,0.0,...,NaN,2630.0,26.0,2.0,1,Central,2001-09-01,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,4.604901e+11,2014-01-01,2014-12-31,NaN,0.000000,33.333332,585.185181,0.0,0.0,0.0,...,Domestic,2640.0,26.0,2.0,2,East,2000-06-14,NaN,NaN,1
1246,4.610112e+11,2014-01-01,2014-12-31,NaN,429.629639,58781.480469,2651.851807,0.0,0.0,0.0,...,Domestic,2620.0,26.0,2.0,3,Central,1997-12-10,NaN,0.242424,0
1247,4.620594e+11,2014-01-01,2014-12-31,NaN,1111.111084,68274.070312,16211.111328,0.0,0.0,0.0,...,Domestic,2630.0,26.0,2.0,1,Central,1996-12-30,NaN,0.222222,0
1248,4.623300e+11,2014-01-01,2014-12-31,NaN,0.000000,17192.591797,7325.925781,0.0,0.0,0.0,...,mix,2611.0,26.0,2.0,2,East,2010-12-15,NaN,0.166667,0


In [136]:
#32 firms defaulted
yo['default'] = (yo['company_existed_2014'] == 1)&(yo['company_not_existed_2015'] == 1)
print(f"# of Defaulted firms: {len(yo.loc[yo['default'] == 1])} firms")

# of Defaulted firms: 32 firms


In [137]:
#the assignment says 56 firms defaulted, but no matter what I do, I never come to this number; therefore, I'm going
#with 32 firms defaulted

In [138]:
#criteria for the holdout sample so I can mask the training sample with !holdout_criteria
holdout_criteria = (data['year'] == 2014) & ((data['sales'] >=1000) & (data['sales']<=10000000))

In [139]:
#holdout sample creation
holdout = data[holdout_criteria]
print(f'Holdout shape: {holdout.shape}')
holdout.head()

Holdout shape: (1037, 48)


,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
969,6538183.0,2014-01-01,2014-12-31,NaN,1018.518494,17022.222656,3040.740723,0.0,0.0,0.0,...,male,Domestic,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
1128,8416055.0,2014-01-01,2014-12-31,NaN,425.925934,105740.742188,6918.518555,0.0,0.0,0.0,...,female,Domestic,2651.0,26.0,2.0,1,Central,1995-08-28,NaN,0.083333
1467,12428378.0,2014-01-01,2014-12-31,NaN,0.000000,0.000000,9277.777344,0.0,0.0,0.0,...,male,Domestic,2660.0,26.0,2.0,2,Central,1991-06-27,NaN,0.083333
1706,17776540.0,2014-01-01,2014-12-31,NaN,48.148148,142296.296875,164237.031250,0.0,0.0,0.0,...,mix,Domestic,2620.0,26.0,2.0,1,Central,1997-07-01,NaN,0.305556
1735,18626760.0,2014-01-01,2014-12-31,NaN,5755.555664,95185.187500,45766.667969,0.0,0.0,0.0,...,male,Domestic,2670.0,26.0,2.0,2,West,2010-11-15,NaN,0.159091


In [140]:
training = data[~holdout_criteria]
training

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
960,6.538183e+06,2005-01-01,2005-12-31,NaN,792.592590,6237.037109,348.148163,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
961,6.538183e+06,2006-01-01,2006-12-31,NaN,196.296295,6007.407227,451.851837,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
962,6.538183e+06,2007-01-01,2007-12-31,NaN,233.333328,9007.407227,2637.037109,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
963,6.538183e+06,2008-01-01,2008-12-31,NaN,422.222229,7707.407227,1651.851807,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
964,6.538183e+06,2009-01-01,2009-12-31,NaN,1525.925903,18196.296875,8714.814453,18.518518,0.0,-18.518518,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287438,4.628231e+11,2011-01-01,2011-12-31,NaN,1462.963013,16007.407227,6392.592773,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.234848
287439,4.628231e+11,2012-01-01,2012-12-31,NaN,359.259247,21662.962891,19481.482422,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.333333
287440,4.628231e+11,2013-01-01,2013-12-31,NaN,3159.259277,15488.888672,21440.740234,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.298611
287442,4.628231e+11,2015-01-01,2015-12-31,NaN,2177.777832,46062.964844,49844.445312,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.562500


In [8]:
holdout.amort.isna().sum()

#x variables = amort, 
#can 

count    1.035000e+03
mean     2.106789e+04
std      9.177529e+04
min      0.000000e+00
25%      4.074074e+02
50%      1.918519e+03
75%      8.962963e+03
max      1.927378e+06
Name: amort, dtype: float64

In [ ]:
from plotnine import *

ggplot(training)